In [34]:
import pandas as pd
from sklearn.model_selection import train_test_split 
import time
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
import pickle
import matplotlib.pyplot as plt

In [35]:
dataset=pd.read_csv("LoanApprovalPrediction.csv")

In [36]:
for columnName in dataset.columns:
    if(dataset[columnName].dtype=='O'):
        print(columnName)
        print(dataset[columnName].value_counts())

Loan_ID
LP002738    1
LP002948    1
LP001577    1
LP001562    1
LP002087    1
           ..
LP001819    1
LP002101    1
LP001119    1
LP002175    1
LP002409    1
Name: Loan_ID, Length: 598, dtype: int64
Gender
Male      487
Female    111
Name: Gender, dtype: int64
Married
Yes    388
No     210
Name: Married, dtype: int64
Education
Graduate        465
Not Graduate    133
Name: Education, dtype: int64
Self_Employed
No     488
Yes    110
Name: Self_Employed, dtype: int64
Property_Area
Semiurban    225
Urban        198
Rural        175
Name: Property_Area, dtype: int64
Loan_Status
Y    411
N    187
Name: Loan_Status, dtype: int64


In [37]:
dataset.Gender=dataset.Gender.replace({"Male" : 1,"Female":0})
dataset.Married=dataset.Married.replace({"Yes" : 1,"No":0})
dataset.Education=dataset.Education.replace({"Graduate" : 1,"Not Graduate":0})
dataset.Self_Employed=dataset.Self_Employed.replace({"Yes" : 1,"No":0})
dataset.Loan_Status=dataset.Loan_Status.replace({"Y":1,"N":0})








In [38]:
from sklearn.preprocessing import LabelEncoder
for Property_Area in dataset.columns:
    le =LabelEncoder()
    dataset['Property_Area']=le.fit_transform(dataset['Property_Area'])

In [39]:
dataset.isnull().sum()

Loan_ID               0
Gender                0
Married               0
Dependents           12
Education             0
Self_Employed         0
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           21
Loan_Amount_Term     14
Credit_History       49
Property_Area         0
Loan_Status           0
dtype: int64

In [40]:
dataset["Dependents"].fillna(dataset["Dependents"].mean(),inplace=True)
dataset["LoanAmount"].fillna(dataset["LoanAmount"].mean(),inplace=True)
dataset["Loan_Amount_Term"].fillna(dataset["Loan_Amount_Term"].mean(),inplace=True)
dataset["Credit_History"].fillna(dataset["Credit_History"].mean(),inplace=True)


In [41]:
dataset.isnull().sum()

Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

In [42]:
no=dataset[dataset.Loan_Status==0]
yes=dataset[dataset.Loan_Status==1]

In [43]:
yes_bal=yes.sample(n=187)

In [44]:
new_dataset=pd.concat([yes_bal,no])

In [65]:
indep_X=new_dataset[['Credit_History','ApplicantIncome',]]

In [66]:
dep_Y=new_dataset[['Loan_Status']]

In [10]:
dep_Y=dataset[['Loan_Status']]

In [67]:
def selectkbest(indep_X,dep_Y,n):
        test = SelectKBest(score_func=chi2, k=n)
        fit1= test.fit(indep_X,dep_Y)
        selectk_features = fit1.transform(indep_X)
        return selectk_features

In [68]:
def split_scalar(indep_X,dep_Y):
        X_train, X_test, y_train, y_test = train_test_split(indep_X, dep_Y, test_size = 0.25, random_state = 0)
        sc = StandardScaler()
        X_train = sc.fit_transform(X_train)
        X_test = sc.transform(X_test)    
        return X_train, X_test, y_train, y_test

In [69]:
def cm_prediction(classifier,X_test):
     y_pred = classifier.predict(X_test)
        
        # Making the Confusion Matrix
     from sklearn.metrics import confusion_matrix
     cm = confusion_matrix(y_test, y_pred)
        
     from sklearn.metrics import accuracy_score 
     from sklearn.metrics import classification_report 
        #from sklearn.metrics import confusion_matrix
        #cm = confusion_matrix(y_test, y_pred)
        
     Accuracy=accuracy_score(y_test, y_pred )
        
     report=classification_report(y_test, y_pred)
     return  classifier,Accuracy,report,X_test,y_test,cm

In [70]:
def logistic(X_train,y_train,X_test):       
        # Fitting K-NN to the Training set
        from sklearn.linear_model import LogisticRegression
        classifier = LogisticRegression(random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm

In [71]:
def svm_linear(X_train,y_train,X_test):
                
        from sklearn.svm import SVC
        classifier = SVC(kernel = 'linear', random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm

In [72]:
def svm_NL(X_train,y_train,X_test):
                
        from sklearn.svm import SVC
        classifier = SVC(kernel = 'rbf', random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm

In [73]:
def Navie(X_train,y_train,X_test):       
        # Fitting K-NN to the Training set
        from sklearn.naive_bayes import GaussianNB
        classifier = GaussianNB()
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm

In [74]:
def knn(X_train,y_train,X_test):
           
        # Fitting K-NN to the Training set
        from sklearn.neighbors import KNeighborsClassifier
        classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm

In [75]:
def Decision(X_train,y_train,X_test):
        
        # Fitting K-NN to the Training set
        from sklearn.tree import DecisionTreeClassifier
        classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm

In [76]:
def random(X_train,y_train,X_test):
        
        # Fitting K-NN to the Training set
        from sklearn.ensemble import RandomForestClassifier
        classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm

In [77]:
def selectk_Classification(acclog,accsvml,accsvmnl,accknn,accnav,accdes,accrf): 
    
    dataframe=pd.DataFrame(index=['ChiSquare'],columns=['Logistic','SVMl','SVMnl','KNN','Navie','Decision','Random'])
    for number,idex in enumerate(dataframe.index):      
        dataframe['Logistic'][idex]=acclog[number]       
        dataframe['SVMl'][idex]=accsvml[number]
        dataframe['SVMnl'][idex]=accsvmnl[number]
        dataframe['KNN'][idex]=accknn[number]
        dataframe['Navie'][idex]=accnav[number]
        dataframe['Decision'][idex]=accdes[number]
        dataframe['Random'][idex]=accrf[number]
    return dataframe

In [78]:
kbest=selectkbest(indep_X,dep_Y,2)

In [79]:
acclog=[]
accsvml=[]
accsvmnl=[]
accknn=[]
accnav=[]
accdes=[]
accrf=[]

In [80]:
X_train, X_test, y_train, y_test=split_scalar(kbest,dep_Y)

In [81]:
classifier,Accuracy,report,X_test,y_test,cm=logistic(X_train,y_train,X_test)
acclog.append(Accuracy)

classifier,Accuracy,report,X_test,y_test,cm=svm_linear(X_train,y_train,X_test)  
accsvml.append(Accuracy)
    
classifier,Accuracy,report,X_test,y_test,cm=svm_NL(X_train,y_train,X_test)  
accsvmnl.append(Accuracy)
    
classifier,Accuracy,report,X_test,y_test,cm=knn(X_train,y_train,X_test)  
accknn.append(Accuracy)
    
classifier,Accuracy,report,X_test,y_test,cm=Navie(X_train,y_train,X_test)  
accnav.append(Accuracy)
    
classifier,Accuracy,report,X_test,y_test,cm=Decision(X_train,y_train,X_test)  
accdes.append(Accuracy)
    
classifier,Accuracy,report,X_test,y_test,cm=random(X_train,y_train,X_test)  
accrf.append(Accuracy)

C:\Users\admin\Anaconda3\envs\MLClustering\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\admin\Anaconda3\envs\MLClustering\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\admin\Anaconda3\envs\MLClustering\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\admin\Anaconda3\envs\MLClustering\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n

In [82]:
result=selectk_Classification(acclog,accsvml,accsvmnl,accknn,accnav,accdes,accrf)

In [83]:
result

,Logistic,SVMl,SVMnl,KNN,Navie,Decision,Random
ChiSquare,0.62766,0.638298,0.62766,0.638298,0.62766,0.606383,0.56383


In [84]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(indep_X,dep_Y,test_size=1/4,random_state=1)

In [85]:
print(x_train)

     Credit_History  ApplicantIncome
67              1.0             7100
156             1.0             4583
332             1.0             2647
35              0.0             1828
157             1.0             1863
..              ...              ...
47              0.0             2645
205             1.0            10000
80              1.0             2395
149             1.0             9166
351             1.0             5167

[280 rows x 2 columns]


In [ ]:
dataset.shape

In [ ]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

In [33]:
from sklearn.svm import SVC  
clf = SVC(kernel='linear') 
clf.fit(x_train,y_train)

NameError: name 'x_train' is not defined

In [ ]:
re=grid.cv_results_
grid_pred=grid.predict(x_test)

In [ ]:
y_pred=classifier.predict(x_test)

In [ ]:
y_pred

In [ ]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,y_pred)

In [ ]:
cm

In [ ]:
from sklearn.metrics import classification_report
clf=classification_report(y_test,y_pred)

In [ ]:
print(clf)

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,classifier.predict_proba(x_test)[:,1])